In [0]:
CREATE OR REPLACE VIEW transacoes_db.pix_baseline_metricas.perfil_de_usuarios AS

WITH PerfilUsuarios AS (
  SELECT
    ep.ano,
    ep.mes,
    f.id AS id_finalidade,
    fi.id AS id_formainiciacao,
    ep.finalidade,
    ep.formainiciacao,
    n_pag.id AS pag_pfpj_id,
    n_rec.id AS rec_pfpj_id,
    ep.natureza,
    ep.pag_idade,
    ep.rec_idade,
    CASE 
      WHEN ep.pag_regiao = 'NORTE' THEN 1
      WHEN ep.pag_regiao = 'NORDESTE' THEN 2
      WHEN ep.pag_regiao = 'SUDESTE' THEN 3
      WHEN ep.pag_regiao = 'SUL' THEN 4
      WHEN ep.pag_regiao = 'CENTRO-OESTE' THEN 5
      ELSE 0
    END AS pag_id_regiao,
    CASE 
      WHEN ep.rec_regiao = 'NORTE' THEN 1
      WHEN ep.rec_regiao = 'NORDESTE' THEN 2
      WHEN ep.rec_regiao = 'SUDESTE' THEN 3
      WHEN ep.rec_regiao = 'SUL' THEN 4
      WHEN ep.rec_regiao = 'CENTRO-OESTE' THEN 5
      ELSE 0
    END AS rec_id_regiao,
    ep.pag_pfpj,
    ep.rec_pfpj,
    ep.pag_regiao,
    ep.rec_regiao,
    CAST(ep.quantidade AS BIGINT) AS quantidade,
    CAST(ep.valor AS DOUBLE) AS valor,
    CASE WHEN ep.quantidade > 0 THEN ep.valor / ep.quantidade ELSE NULL END AS ticket_individual
  FROM estatisticas_pix.gold.estatisticas_transacoes_pix ep
  LEFT JOIN transacoes_db.copper.finalidade_pix f 
    ON ep.finalidade = f.nome
  LEFT JOIN transacoes_db.copper.tipos_iniciacao_pix fi 
    ON ep.formainiciacao = fi.nome
  LEFT JOIN transacoes_db.copper.naturezas n_pag
    ON ep.pag_pfpj = n_pag.sigla
  LEFT JOIN transacoes_db.copper.naturezas n_rec
    ON ep.rec_pfpj = n_rec.sigla
  WHERE ep.pag_regiao IS NOT NULL
    AND ep.rec_regiao IS NOT NULL
    AND ep.pag_regiao <> 'Nao informado'
    AND ep.rec_regiao <> 'Nao informado'
    AND ep.pag_idade IS NOT NULL
    AND ep.rec_idade IS NOT NULL
    AND ep.pag_idade NOT IN ('Nao se aplica', 'Nao informado')
    AND ep.rec_idade NOT IN ('Nao se aplica', 'Nao informado')
    AND ep.quantidade > 0
)

SELECT
  ano,
  mes,
  id_finalidade,
  id_formainiciacao,
  finalidade,
  formainiciacao,
  natureza,
  pag_idade,
  rec_idade,
  pag_pfpj,
  rec_pfpj,
  pag_pfpj_id,
  rec_pfpj_id,
  pag_id_regiao,
  rec_id_regiao,
  pag_regiao,
  rec_regiao,
  SUM(valor) AS total_valor,
  SUM(quantidade) AS total_quantidade,
  COUNT(*) AS total_transacoes,
  ROUND(SUM(valor) / NULLIF(SUM(quantidade), 0), 2) AS ticket_medio,
  AVG(ticket_individual) AS ticket_medio_aritmetico,
  percentile_approx(ticket_individual, 0.5) AS ticket_mediana,
  MIN(ticket_individual) AS ticket_minimo,
  MAX(ticket_individual) AS ticket_maximo,
  percentile_approx(ticket_individual, 0.25) AS ticket_p25,
  percentile_approx(ticket_individual, 0.75) AS ticket_p75
FROM PerfilUsuarios
GROUP BY
  ano,
  mes,
  id_finalidade,
  id_formainiciacao,
  finalidade,
  formainiciacao,
  natureza,
  pag_idade,
  rec_idade,
  pag_pfpj,
  rec_pfpj,
  pag_pfpj_id,
  rec_pfpj_id,
  pag_regiao,
  rec_regiao,
  pag_id_regiao,
  rec_id_regiao
;